In [14]:
import requests, json, pandas, itertools
import numpy as np
import time

print("hello is it me you're looking for?")

def send_req_name(name):
    url = 'http://musicbrainz.org/ws/2/artist/?query=artist:' + name
    headers = {'Accept': 'application/json'}
    r = requests.get(url, headers=headers)
    page = r.json()
    return page

def send_req_set(num, id):
    url = 'https://api.setlist.fm/rest/1.0/artist/' + id + '/setlists/?p=' + num
    headers = {'Accept': 'application/json', 'x-api-key': 'd65d2f04-4d2a-4354-b6dd-45bedf61cde1'}
    r = requests.get(url, headers=headers)
    page = r.json()
    return page

#load artists names
names = pandas.read_csv("spotify.csv")

t = names["Artist"]

artists = []

#remove duplicates
for i in t:
       if i not in artists:
          artists.append(i)

# Not used for various reasons
# "f69e3f10-4f8d-4847-89b0-5805a9fa9f7b", 7924f0b1-8a8c-4cee-a4c9-7f63a822496a, 7590d967-65a7-4dae-bff8-741a830938db,
# d161be49-5c93-4467-b776-7d0dc2cd2d80, 02861daf-f923-49fb-90e3-3ec44a600433, 20a95f96-78b6-4880-846d-dd17b5ee92ac,
#61e8c00d-72b0-4a23-b0aa-5fe90a1f9968

# ids = []

ids = ["9fff2f8a-21e6-47de-a2b8-7f449929d43f","0221734a-35a8-40f7-b5c5-2f61a200bc7c","c8b03190-306c-4120-bb0b-6f2ebfc06ea9",
 "c78c3026-426b-4580-ab23-08490bb8b515","91a81925-92f9-4fc9-b897-93cf01226282","f0d574e0-638c-463a-92dd-33aa9339cffb",
 "f6af669a-56ea-448a-a044-de76181ada33","985f7e6f-0a7e-4de7-b9ec-a5dac63cb2f7","875203e1-8e58-4b86-8dcb-7190faf411c5",
 "6247d69d-426f-440f-97e0-74a9a0b00358","942a9807-9c1a-4a0e-a285-1fde2c5be9d1","afb680f2-b6eb-4cd7-a70b-a63b25c763d5",
 "0ab49580-c84f-44d4-875f-d83760ea2cfe","6ca721e3-f89c-44a3-b786-c8a84a601712","31445b1e-96f3-4c7f-84ec-dd02e647e690",
 "37b2cb82-ef79-4d46-a184-a549450aa231","373f5bcf-f1ce-4515-83ed-0b8ed636bc14","3e1f2ee4-16be-4406-bf18-6173840cf2b1",
 "f4fdbb4c-e4b7-47a0-b83b-d91bbfcfa387","56f62fe8-d3a0-46e0-b5df-a64a8b4b4274","a6c6897a-7415-4f8d-b5a5-3a5e05f3be67",
 "502b3244-0f7d-470c-99e9-b17fe6bcaabf","1d6b30fe-1565-4a6c-b9e7-3ffa76f6514f","494e8d09-f85b-4543-892f-a5096aed1cd4",
 "97e69730-3791-423b-9770-287261588854","8837b875-3689-4646-b3fd-d8b53815c7a8","a39acc4a-985e-4173-a9ed-f325f2d3bc1c",
 "2f548675-008d-4332-876c-108b0c7ab9c5","6703863e-2dda-4079-90ab-ef7b72d14881","6ca4886a-8f77-4133-b404-46fb3908e24a",
 "75a72702-a5ef-4513-bca5-c5b944903546","373a4c98-a46b-48e4-86ec-f6ca65b4f438","75b7ef51-0b2c-4e86-978a-b8a98095eed7",
 "b7d92248-97e3-4450-8057-6fe06738f735","7fb57fba-a6ef-44c2-abab-2fa3bdee607e","73e5e69d-3554-40d8-8516-00cb38737a1c",
 "75be165a-ad83-4d12-bd28-f589a15c479f","1a9cc1a5-51f1-4deb-91fe-da866f6cc6ce","dacc2d64-7e59-42a8-87a9-10c53919aff2",
 "6eb87ebd-43bd-4bdc-adbd-48744947f692","2437980f-513a-44fc-80f1-b90d9d7fcf8f","086c1936-9eaa-4091-b140-ad33a51a3b71",
 "c211af3d-65e0-46a7-ac49-6c84837a0d16","05b75ee5-98cd-430e-b1ec-ec0f61fdabda",
 "ac9a487a-d9d2-4f27-bb23-0f4686488345","fbe054ec-a143-4101-9e9e-64abc5ff5ac9","2bd8101a-11c3-42d5-a9bd-6a3b26445f7e",
 "db48610c-6d8f-4d10-90ca-c4d14bb1a8b1","5ec1ecc6-4535-4911-a27a-e26343f4a189",
 "01809552-4f87-45b0-afff-2c6f0730a3be","4aeb9b20-9c99-4dad-9f6b-4f16e831c700","02f44a81-34f0-4f52-9ab7-c3cfcdb60333",
 "a668565b-41b8-44a8-9d5c-8e609dbf69fc","c1708d03-8a66-46eb-848e-fe0d233ffb39","5df148ae-78eb-495a-8a0b-d16f3105992d",
 "592a3b6d-c42b-4567-99c9-ecf63bd66499","d8fd8d9b-473b-4f06-83c8-869b1bb9de89","b1e26560-60e5-4236-bbdb-9aa5a8d5ee19",
 "aa5a6061-e60d-4a58-b9f6-c09de390bb2d","2eb55bd9-cfbf-44fd-a5da-92dd825284c7","197450cd-0124-4164-b723-3c22dd16494d",
 "30ae6581-15ec-407b-b192-5f7b7ea214b2","4bafe97c-81d4-4a8e-b54b-7f6e60ea7abe",
 "8fbcd70f-bf1e-4624-b94a-fcff6b0c637e","e4a51f17-a57b-47b1-b37b-f552d0f8e9e6","f23ef341-33bd-47c1-b83c-846a78581f05",
 "79c67eb5-3851-4b28-bfe0-3adfcd6f58d7","d570bb31-b9ca-45a4-a2d9-11b186d9c159","cc2c9c3c-b7bc-4b8b-84d8-4fbd8779e493",
 "611700cf-27f0-4dc9-ae80-c513a767853e","09a6607d-4d1e-42e8-8b67-5af2b5156f86","23101701-cecf-4c9b-9b4c-d6a249eac118",
"31bcadcc-e1da-4cad-bec8-2f4f1d41b095","e565610b-9a8f-451d-b594-ea658d76f725",
 "134e6410-6954-45d1-bd4a-0f2d2ad5471d","596ffa74-3d08-44ef-b113-765d43d12738","e22391dd-9c76-4f02-88b5-35d699967868",
 "164f0d73-1234-4e2c-8743-d77bf2191051","744b52c8-509b-4451-abfd-a17d18d4bd1d","0182474b-8a1d-46d0-9d0f-e6df353335ea",
 "1036b808-f58c-4a3e-b461-a2c4492ecf1b","5b64e344-b2fb-47bd-bd7b-4c793b17d472",
 "525f1f1c-03f0-4bc8-8dfd-e7521f87631b","9ddf4b19-dd14-45d9-b056-49541b16dc80","dbc26631-5e61-4ba2-84ff-e1423a1c288a",
 "d4caad3c-c5c4-44a4-b90c-900650965ba8",
  "db8fad3a-e131-47a1-8782-c2ee93708cdd",
 "55e6074f-ef78-4ec3-8fff-bd1b8cc8c14a","17678771-5799-4017-851a-319f25b6948d","163218c2-9e0f-4d2d-aecb-ebbdef6aab70",
 "b9348d59-b91b-423f-847b-8db155a0653b","e0140a67-e4d1-4f13-8a01-364355bee46e",
 "bf24ca37-25f4-4e34-9aec-460b94364cfc","1c009a08-74d1-4682-bc56-8bfbe9e66b62","35fd8d42-b4a6-4414-9827-8766bd0daa3c",
 "723a441f-26d5-428c-8669-0f8b7276920f","c234fa42-e6a6-443e-937e-2f4b073538a3","c3bba486-b24b-4a39-bc4c-b0085950afcd",
 "e0e1db18-f7ba-4dee-95ff-7ae8cf545460","124cec07-44f1-4a99-9aaa-96a5bb9ec0bd","dff0d392-4cd5-4052-9fbb-f485df3891e5",
 "b02ec63e-e78d-43e5-a594-462e95036e8d","b9f0dfa1-7cf1-4e3c-8770-c8bc660c000a","322fe370-6ff9-4fad-8c18-a00aa41ba2d9",
 "f45da029-7b00-4bf3-962c-0eeb20309cc4","0c41b966-12e8-4c9b-87ea-af47798987b6","8e024f18-304f-430d-b1ba-ff6307743b15",
 "7ecd398b-3278-4a7c-8ed9-7c3e786d9a02","715d6b2d-9b1f-4615-8e7c-71cf5e9bcde3","d43c0ca4-6abf-4ce3-995b-03546cf31cd6",
 "298f4089-1948-44b5-b5e8-d5381f42362f"]


#get ids for artisst names
# for artist in artists:
#     print(artist)
#     page = send_req_name(artist)
#     list = page["artists"]
#     id = list[0]["id"]
#     print(id)
#     ids.append(id)
#     time.sleep(1) 

# print("got all ids")

#ids = ["9fff2f8a-21e6-47de-a2b8-7f449929d43f", "8e494408-8620-4c6a-82c2-c2ca4a1e4f12", "cc197bad-dc9c-440d-a5b5-d52ba2e14234"]


#create empty data table
d = {'venues': []}
df = pandas.DataFrame(d)
df['cities'] = []
df['artists'] = []

#loop through each artist id
for id in ids:
    print(id)
    i = 1
    keep_going = True
    count = 0
    cities = []
    venues = []
    artists2 = []
    artist_name = ""
    
    #keep counting up by page until you run a page is not returned
    while(keep_going):
        #pull artists setlist page
        page = send_req_set(str(i), id)
        if('code' in page):
            keep_going = False
        else:
            setlist = page["setlist"]
            for sets in setlist:
                #add page data to sets
                artist2 = sets["artist"]
                artist_name = artist2["name"]
                venue = sets["venue"]
                venue_name = venue["name"]
                city = venue["city"]
                name = city["name"]
                count = count + 1
                venues.append(venue_name)
                cities.append(name)
            i = i + 1

    #add artist to dataframe
    artists2 = np.repeat(artist_name, len(cities)).reshape(len(cities),1)
    d = {'venues': venues}
    dft = pandas.DataFrame(d)
    dft['cities'] = cities
    dft['artists'] = artists2
    df = pandas.concat([df,dft])
    
print(df)
df.to_csv("data.csv")


hello is it me you're looking for?
9fff2f8a-21e6-47de-a2b8-7f449929d43f
0221734a-35a8-40f7-b5c5-2f61a200bc7c
c8b03190-306c-4120-bb0b-6f2ebfc06ea9
c78c3026-426b-4580-ab23-08490bb8b515
91a81925-92f9-4fc9-b897-93cf01226282
f0d574e0-638c-463a-92dd-33aa9339cffb
f6af669a-56ea-448a-a044-de76181ada33
985f7e6f-0a7e-4de7-b9ec-a5dac63cb2f7
875203e1-8e58-4b86-8dcb-7190faf411c5
6247d69d-426f-440f-97e0-74a9a0b00358
942a9807-9c1a-4a0e-a285-1fde2c5be9d1
afb680f2-b6eb-4cd7-a70b-a63b25c763d5
0ab49580-c84f-44d4-875f-d83760ea2cfe
6ca721e3-f89c-44a3-b786-c8a84a601712
31445b1e-96f3-4c7f-84ec-dd02e647e690
37b2cb82-ef79-4d46-a184-a549450aa231
373f5bcf-f1ce-4515-83ed-0b8ed636bc14
3e1f2ee4-16be-4406-bf18-6173840cf2b1
f4fdbb4c-e4b7-47a0-b83b-d91bbfcfa387
56f62fe8-d3a0-46e0-b5df-a64a8b4b4274
a6c6897a-7415-4f8d-b5a5-3a5e05f3be67
502b3244-0f7d-470c-99e9-b17fe6bcaabf
1d6b30fe-1565-4a6c-b9e7-3ffa76f6514f
494e8d09-f85b-4543-892f-a5096aed1cd4
97e69730-3791-423b-9770-287261588854
8837b875-3689-4646-b3fd-d8b53815c7a8
a39